<a href="https://colab.research.google.com/github/2403a54128/NLP/blob/main/NLP_ASS(6_3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
corpus = [
    "virat scored century in match",
    "bjp won in elections",
    "Bumra took 5 wicket in a match",
    "Congress form state government"
]

In [2]:
documents = corpus
print(documents)


['virat scored century in match', 'bjp won in elections', 'Bumra took 5 wicket in a match', 'Congress form state government']


In [3]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

processed_docs = []

for doc in documents:

    # Clean text (remove numbers & special chars)
    doc = re.sub(r'[^a-zA-Z]', ' ', doc)

    # Lowercase
    doc = doc.lower()

    # Tokenization
    words = doc.split()

    # Stopword removal + Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Rejoin
    processed_docs.append(" ".join(words))

print(processed_docs)

['virat scored century match', 'bjp election', 'bumra took wicket match', 'congress form state government']


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(processed_docs)


In [7]:
import pandas as pd

bow_df = pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names_out())
print(bow_df)


   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [8]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=2, random_state=42)
lda.fit(bow)


LatentDirichletAllocation(n_components=2, random_state=42)

In [9]:
words = vectorizer.get_feature_names_out()

for i, topic in enumerate(lda.components_):
    print(f"\nTopic {i+1}:")
    print([words[index] for index in topic.argsort()[-5:]])



Topic 1:
['election', 'state', 'congress', 'government', 'form']

Topic 2:
['took', 'scored', 'century', 'virat', 'match']


In [10]:
#task-2



In [11]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

import nltk
from nltk.corpus import stopwords
import re


In [13]:
df = pd.read_csv("/content/arxiv_data.csv", on_bad_lines='warn', engine='python')   # change dataset name

print(df.head())

                                              titles  \
0  Survey on Semantic Stereo Matching / Semantic ...   
1  FUTURE-AI: Guiding Principles and Consensus Re...   
2  Enforcing Mutual Consistency of Hard Regions f...   
3  Parameter Decoupling Strategy for Semi-supervi...   
4  Background-Foreground Segmentation for Interio...   

                                           summaries  \
0  Stereo matching is one of the widely used tech...   
1  The recent advancements in artificial intellig...   
2  In this paper, we proposed a novel mutual cons...   
3  Consistency training has proven to be an advan...   
4  To ensure safety in automated driving, the cor...   

                         terms  
0           ['cs.CV', 'cs.LG']  
1  ['cs.CV', 'cs.AI', 'cs.LG']  
2           ['cs.CV', 'cs.AI']  
3                    ['cs.CV']  
4           ['cs.CV', 'cs.LG']  


In [15]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Ensure text is a string before processing
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

# Assuming you want to clean the 'summaries' column from the df DataFrame
# texts = df['summaries'].apply(clean_text) # Original line
texts = df['summaries'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import re

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z ]', '', text)
    return text

texts_clean = texts.apply(preprocess)

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english')
bow_matrix = vectorizer.fit_transform(texts_clean)


In [17]:
print(texts_clean.isnull().sum())
print(texts_clean.head())


0
0    stereo matching one widely used techniques inf...
1    recent advancements artificial intelligence ai...
2    paper proposed novel mutual consistency networ...
3    consistency training proven advanced semisuper...
4    ensure safety automated driving correct percep...
Name: summaries, dtype: object


In [18]:
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print(bow_df.head())


   aa  aafor  aasegfor  aatr  abackgroundwhich  abalanced  aball  abandonthe  \
0   0      0         0     0                 0          0      0           0   
1   0      0         0     0                 0          0      0           0   
2   0      0         0     0                 0          0      0           0   
3   0      0         0     0                 0          0      0           0   
4   0      0         0     0                 0          0      0           0   

   abasic  abatch  ...  znn  zoclip  zoclipto  zones  zoom  zooming  zoonotic  \
0       0       0  ...    0       0         0      0     0        0         0   
1       0       0  ...    0       0         0      0     0        0         0   
2       0       0  ...    0       0         0      0     0        0         0   
3       0       0  ...    0       0         0      0     0        0         0   
4       0       0  ...    0       0         0      0     0        0         0   

   zoos  zsl  zurich  
0     0  

In [19]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(
    n_components=5,   # number of topics
    random_state=42
)

lda_model.fit(bow_matrix)


LatentDirichletAllocation(n_components=5, random_state=42)

In [20]:
def display_topics(model, feature_names, no_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic", topic_idx+1)
        print([feature_names[i] for i in topic.argsort()[:-no_words - 1:-1]])

display_topics(lda_model, vectorizer.get_feature_names_out(), 10)



Topic 1
['segmentation', 'image', 'images', 'method', 'model', 'data', 'learning', 'proposed', 'results', 'network']

Topic 2
['learning', 'graph', 'data', 'representations', 'representation', 'tasks', 'model', 'propose', 'information', 'methods']

Topic 3
['learning', 'segmentation', 'image', 'representation', 'methods', 'method', 'images', 'network', 'deep', 'propose']

Topic 4
['segmentation', 'image', 'learning', 'data', 'model', 'method', 'images', 'results', 'models', 'performance']

Topic 5
['learning', 'data', 'image', 'segmentation', 'training', 'model', 'images', 'method', 'methods', 'using']


In [21]:
lda_topics = lda_model.transform(bow_matrix)

df['LDA_Topic'] = lda_topics.argmax(axis=1)

print(df[['summaries','LDA_Topic']].head())

                                           summaries  LDA_Topic
0  Stereo matching is one of the widely used tech...          3
1  The recent advancements in artificial intellig...          1
2  In this paper, we proposed a novel mutual cons...          3
3  Consistency training has proven to be an advan...          0
4  To ensure safety in automated driving, the cor...          3


In [ ]:
task-3


In [22]:
import pandas as pd

# Corpus
documents = [
    "Virat scored century in match",
    "BJP won in elections",
    "Bumra took 5 wicket in a match",
    "Congress form state government"
]

df = pd.DataFrame(documents, columns=["Text"])
print(df)


                             Text
0   Virat scored century in match
1            BJP won in elections
2  Bumra took 5 wicket in a match
3  Congress form state government


In [23]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Added to download the missing resource

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Lowercase
    text = text.lower()

    # Remove numbers & punctuation
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Tokenize
    words = word_tokenize(text)

    # Remove stopwords & Lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    # Rejoin
    return " ".join(words)

df["Clean_Text"] = df["Text"].apply(preprocess)

print(df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...


                             Text                      Clean_Text
0   Virat scored century in match      virat scored century match
1            BJP won in elections                    bjp election
2  Bumra took 5 wicket in a match         bumra took wicket match
3  Congress form state government  congress form state government


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(df["Clean_Text"])


In [25]:
bow_df = pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names_out())
print(bow_df)


   bjp  bumra  century  congress  election  form  government  match  scored  \
0    0      0        1         0         0     0           0      1       1   
1    1      0        0         0         1     0           0      0       0   
2    0      1        0         0         0     0           0      1       0   
3    0      0        0         1         0     1           1      0       0   

   state  took  virat  wicket  
0      0     0      1       0  
1      0     0      0       0  
2      0     1      0       1  
3      1     0      0       0  


In [26]:
from sklearn.decomposition import NMF

num_topics = 2

nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(bow)


NMF(n_components=2, random_state=42)

In [27]:
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"\nTopic {topic_idx + 1}:")
    print([feature_names[i] for i in topic.argsort()[-5:]])



Topic 1:
['wicket', 'scored', 'took', 'virat', 'match']

Topic 2:
['election', 'government', 'congress', 'form', 'state']


In [28]:
topic_values = nmf_model.transform(bow)

df["Topic"] = topic_values.argmax(axis=1) + 1

print("\nFinal Topic Assignment:")
print(df[["Text", "Topic"]])



Final Topic Assignment:
                             Text  Topic
0   Virat scored century in match      1
1            BJP won in elections      2
2  Bumra took 5 wicket in a match      1
3  Congress form state government      2


In [ ]:
task-4


In [34]:
import pandas as pd

# Load dataset (change path to your downloaded Kaggle file)
df = pd.read_csv("/content/arxiv_data.csv", on_bad_lines='warn', engine='python')   # example file name

# Display columns
print(df.columns)

# Assume abstract column contains text
corpus = df['summaries'].dropna().tolist()

print("Total Documents:", len(corpus))

Index(['titles', 'summaries', 'terms'], dtype='object')
Total Documents: 7181


/tmp/ipython-input-839692497.py:4: ParserWarning: Skipping line 7183: unexpected end of data

  df = pd.read_csv("/content/arxiv_data.csv", on_bad_lines='warn', engine='python')   # example file name


In [35]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Lowercase
    text = text.lower()

    # Remove numbers & special characters
    text = re.sub(r'[^a-zA-Z ]', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Rejoin
    return " ".join(tokens)

clean_corpus = [preprocess(doc) for doc in corpus]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=3000)

bow_matrix = vectorizer.fit_transform(clean_corpus)


In [37]:
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print(bow_df.head())


   abdominal  ability  ablation  able  abnormal  absence  absolute  abstract  \
0          0        0         0     0         0        0         0         0   
1          0        0         0     0         0        0         0         0   
2          0        0         0     0         0        0         0         0   
3          0        1         0     0         0        0         0         0   
4          0        0         0     0         0        0         0         0   

   abstraction  abundant  ...  wsod  xray  year  yet  yield  yielding  yolo  \
0            0         0  ...     0     0     0    0      0         0     0   
1            0         0  ...     0     0     0    0      0         0     0   
2            0         0  ...     0     0     0    0      0         0     0   
3            0         0  ...     0     0     0    0      0         0     0   
4            0         0  ...     0     0     1    0      0         0     0   

   yolov  zero  zeroshot  
0      0     0   

In [38]:
from sklearn.decomposition import NMF

num_topics = 5

nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_model.fit(bow_matrix)


NMF(n_components=5, random_state=42)

In [39]:
feature_names = vectorizer.get_feature_names_out()

def display_topics(model, feature_names, num_words=10):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx+1}:")
        print([feature_names[i] for i in topic.argsort()[:-num_words-1:-1]])

display_topics(nmf_model, feature_names)



Topic 1:
['learning', 'representation', 'data', 'method', 'task', 'graph', 'feature', 'approach', 'deep', 'learn']

Topic 2:
['image', 'segmentation', 'method', 'result', 'using', 'algorithm', 'proposed', 'generation', 'approach', 'based']

Topic 3:
['object', 'detection', 'method', 'detector', 'feature', 'dataset', 'point', 'performance', 'box', 'propose']

Topic 4:
['network', 'neural', 'feature', 'segmentation', 'deep', 'method', 'graph', 'task', 'proposed', 'architecture']

Topic 5:
['model', 'data', 'training', 'generative', 'deep', 'show', 'performance', 'domain', 'trained', 'datasets']


In [41]:
topic_values = nmf_model.transform(bow_matrix)

df['Dominant_Topic'] = topic_values.argmax(axis=1)

print(df[['summaries', 'Dominant_Topic']].head())

                                           summaries  Dominant_Topic
0  Stereo matching is one of the widely used tech...               3
1  The recent advancements in artificial intellig...               1
2  In this paper, we proposed a novel mutual cons...               4
3  Consistency training has proven to be an advan...               4
4  To ensure safety in automated driving, the cor...               1


In [70]:
#Task-5(sample data-1 using TF.IDF)

In [71]:
import pandas as pd

# Load Kaggle dataset (change file name/path)
df = pd.read_csv("/content/arxiv_data.csv", on_bad_lines='warn', engine='python')

# Check columns
print(df.columns)

# Take abstracts as corpus
corpus = df['summaries'].dropna().tolist()

print("Total Documents:", len(corpus))

Index(['titles', 'summaries', 'terms'], dtype='object')
Total Documents: 51774


In [72]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):

    # Lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub(r'[^a-zA-Z ]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Stopword removal + Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Rejoin
    return " ".join(tokens)

clean_corpus = [preprocess(doc) for doc in corpus]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=3000)

tfidf_matrix = tfidf_vectorizer.fit_transform(clean_corpus)


In [49]:
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

print(tfidf_df.head())


    ability  ablation  able  abnormal  absence  absolute  abstract  \
0  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   
1  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   
2  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   
3  0.080848       0.0   0.0       0.0      0.0       0.0       0.0   
4  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   

   abstraction  abundant   ac  ...  xray     year  yet  yield  yielding  yolo  \
0          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
1          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
2          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
3          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
4          0.0       0.0  0.0  ...   0.0  0.07296  0.0    0.0       0.0   0.0   

   yolov  zero  zeroshot  zsl  
0    0.0   0.0       0.0  0.0  
1    0.0   0.0       0.0  0.

In [75]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 5

lda_model = LatentDirichletAllocation(
    n_components=num_topics,
    random_state=42
)

lda_model.fit(tfidf_matrix)


LatentDirichletAllocation(n_components=5, random_state=42)

In [76]:
feature_names = tfidf_vectorizer.get_feature_names_out()

def display_topics(model, feature_names, num_words=10):

    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx+1}:")
        print([feature_names[i] for i in topic.argsort()[:-num_words-1:-1]])

display_topics(lda_model, feature_names)



Topic 1:
['image', 'feature', 'model', 'video', 'network', 'object', 'method', 'attention', 'visual', 'task']

Topic 2:
['image', 'model', 'data', 'network', 'segmentation', 'learning', 'method', 'deep', 'time', 'neural']

Topic 3:
['object', 'depth', 'detection', 'point', 'image', 'scene', 'method', 'estimation', 'camera', 'cloud']

Topic 4:
['graph', 'node', 'data', 'representation', 'network', 'learning', 'method', 'point', 'model', 'feature']

Topic 5:
['learning', 'model', 'policy', 'algorithm', 'reinforcement', 'agent', 'rl', 'problem', 'data', 'method']


In [74]:
topic_values = lda_model.transform(tfidf_matrix)

df['Dominant_Topic'] = topic_values.argmax(axis=1)

print(df[['summaries', 'Dominant_Topic']].head())

                                           summaries  Dominant_Topic
0  Stereo matching is one of the widely used tech...               0
1  The recent advancements in artificial intellig...               2
2  In this paper, we proposed a novel mutual cons...               1
3  Consistency training has proven to be an advan...               1
4  To ensure safety in automated driving, the cor...               1


In [77]:
#task-6(using nmf)

In [55]:
import pandas as pd

# Load Kaggle dataset (change file name/path)
df = pd.read_csv("/content/arxiv_data.csv", on_bad_lines='warn', engine='python')

# Check columns
print(df.columns)

# Take abstracts as corpus
corpus = df['summaries'].dropna().tolist()

print("Total Documents:", len(corpus))

Index(['titles', 'summaries', 'terms'], dtype='object')
Total Documents: 51774


In [56]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):

    # Lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub(r'[^a-zA-Z ]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Stopword removal + Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Rejoin
    return " ".join(tokens)

clean_corpus = [preprocess(doc) for doc in corpus]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [57]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=3000)

bow_matrix = vectorizer.fit_transform(clean_corpus)


In [58]:
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

print(tfidf_df.head())


    ability  ablation  able  abnormal  absence  absolute  abstract  \
0  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   
1  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   
2  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   
3  0.080848       0.0   0.0       0.0      0.0       0.0       0.0   
4  0.000000       0.0   0.0       0.0      0.0       0.0       0.0   

   abstraction  abundant   ac  ...  xray     year  yet  yield  yielding  yolo  \
0          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
1          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
2          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
3          0.0       0.0  0.0  ...   0.0  0.00000  0.0    0.0       0.0   0.0   
4          0.0       0.0  0.0  ...   0.0  0.07296  0.0    0.0       0.0   0.0   

   yolov  zero  zeroshot  zsl  
0    0.0   0.0       0.0  0.0  
1    0.0   0.0       0.0  0.

In [59]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 5

lda_model = LatentDirichletAllocation(
    n_components=num_topics,
    random_state=42
)

lda_model.fit(tfidf_matrix)


LatentDirichletAllocation(n_components=5, random_state=42)

In [60]:
feature_names = tfidf_vectorizer.get_feature_names_out()

def display_topics(model, feature_names, num_words=10):

    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx+1}:")
        print([feature_names[i] for i in topic.argsort()[:-num_words-1:-1]])

display_topics(lda_model, feature_names)



Topic 1:
['graph', 'model', 'data', 'network', 'learning', 'method', 'neural', 'algorithm', 'problem', 'representation']

Topic 2:
['image', 'data', 'model', 'learning', 'segmentation', 'method', 'network', 'domain', 'deep', 'training']

Topic 3:
['learning', 'policy', 'agent', 'reinforcement', 'rl', 'graph', 'algorithm', 'environment', 'reward', 'task']

Topic 4:
['object', 'feature', 'point', 'detection', 'cloud', 'segmentation', 'method', 'image', 'network', 'attention']

Topic 5:
['image', 'video', 'depth', 'method', 'object', 'scene', 'model', 'network', 'face', 'motion']
